In [37]:
import pandas as pd

# Load restaurant data
df_restaurants = pd.read_csv('combined_restaurants_cleaned.csv')

# Load S0101 Census data (Age and Sex)
df_census_s0101 = pd.read_csv('ACSST5Y2023.S0101-Data.csv', skiprows=1)

# Optional: Load DP05 Census data (Demographic Estimates)
df_census_dp05 = pd.read_csv('ACSDP5Y2023.DP05-Data.csv', skiprows=1)



/var/folders/08/wxl3s97n5hq2wl2zk18pft740000gn/T/ipykernel_27227/1715449768.py:7: DtypeWarning: Columns (3,5,11,13,15,17,23,25,41,43,45,47,49,53,61,63,155,157,163,165,167,169,175,177,193,195,197,199,201,205,213,215,307,309,315,317,319,321,327,329,345,347,349,351,353,357,365,367) have mixed types. Specify dtype option on import or set low_memory=False.
  df_census_s0101 = pd.read_csv('ACSST5Y2023.S0101-Data.csv', skiprows=1)
/var/folders/08/wxl3s97n5hq2wl2zk18pft740000gn/T/ipykernel_27227/1715449768.py:10: DtypeWarning: Columns (3,5,7,11,17,19,21,23,25,31,39,43,49,51,53,55,59,61,63,67,137,151,153,163) have mixed types. Specify dtype option on import or set low_memory=False.
  df_census_dp05 = pd.read_csv('ACSDP5Y2023.DP05-Data.csv', skiprows=1)


In [38]:
# Extract ZIP code from restaurant address
df_restaurants['zip'] = df_restaurants['address'].str.extract(r'(\d{5})')

# Preview ZIP codes in both datasets
print(df_restaurants[['address', 'zip']].head())
print(df_census_s0101.columns)


                                            address    zip
0  145 W 53rd St, New York, NY 10019, United States  10019
1    9 W 53rd St, New York, NY 10019, United States  10019
2   1114 6th Ave, New York, NY 10036, United States  10036
3  147 W 43rd St, New York, NY 10036, United States  10036
4   20 W 40th St, New York, NY 10018, United States  10018
Index(['Geography', 'Geographic Area Name',
       'Estimate!!Total!!Total population',
       'Margin of Error!!Total!!Total population',
       'Estimate!!Total!!Total population!!AGE!!Under 5 years',
       'Margin of Error!!Total!!Total population!!AGE!!Under 5 years',
       'Estimate!!Total!!Total population!!AGE!!5 to 9 years',
       'Margin of Error!!Total!!Total population!!AGE!!5 to 9 years',
       'Estimate!!Total!!Total population!!AGE!!10 to 14 years',
       'Margin of Error!!Total!!Total population!!AGE!!10 to 14 years',
       ...
       'Margin of Error!!Percent Female!!Total population!!SUMMARY INDICATORS!!Age dependen

In [39]:
df_census_s0101['zip'] = df_census_s0101['Geographic Area Name'].str.extract(r'(\d{5})')


In [40]:
df_census_subset = df_census_s0101[['zip', 'Estimate!!Total!!Total population']].copy()
df_census_subset.rename(columns={'Estimate!!Total!!Total population': 'population'}, inplace=True)


In [41]:
df_merged = df_restaurants.merge(df_census_subset, on='zip', how='left')

In [42]:
print(df_merged[['name', 'zip', 'population']].head())
print(df_merged['population'].isnull().mean())  # Check how many rows have missing census data


                           name    zip  population
0           La Grande Boucherie  10019     44276.0
1                    The Modern  10019     44276.0
2                STK Steakhouse  10036     30589.0
3              Tony's Di Napoli  10036     30589.0
4  La Pecora Bianca Bryant Park  10018      8764.0
0.004557885141294439


In [43]:
df_census_subset = df_census_s0101[[
    'Geographic Area Name',
    'Estimate!!Total!!Total population',
    'Estimate!!Percent Female!!Total population',
    'Estimate!!Total!!Total population!!AGE!!Under 5 years',
    'Estimate!!Total!!Total population!!AGE!!5 to 9 years',
    'Estimate!!Total!!Total population!!AGE!!10 to 14 years'
]].copy()

# Extract ZIP code from 'Geographic Area Name'
df_census_subset['zip'] = df_census_subset['Geographic Area Name'].str.extract(r'(\d{5})')

# Rename columns for easier use
df_census_subset.rename(columns={
    'Estimate!!Total!!Total population': 'population',
    'Estimate!!Percent Female!!Total population': 'percent_female',
    'Estimate!!Total!!Total population!!AGE!!Under 5 years': 'age_under_5',
    'Estimate!!Total!!Total population!!AGE!!5 to 9 years': 'age_5_9',
    'Estimate!!Total!!Total population!!AGE!!10 to 14 years': 'age_10_14'
}, inplace=True)


In [44]:
df_merged = df_restaurants.merge(df_census_subset, on='zip', how='left')


In [45]:
print(df_merged[['name', 'zip', 'population', 'percent_female', 'age_under_5', 'age_5_9', 'age_10_14']].head())


                           name    zip  population percent_female  \
0           La Grande Boucherie  10019     44276.0            (X)   
1                    The Modern  10019     44276.0            (X)   
2                STK Steakhouse  10036     30589.0            (X)   
3              Tony's Di Napoli  10036     30589.0            (X)   
4  La Pecora Bianca Bryant Park  10018      8764.0            (X)   

   age_under_5  age_5_9  age_10_14  
0       1383.0    753.0      369.0  
1       1383.0    753.0      369.0  
2        551.0    649.0      585.0  
3        551.0    649.0      585.0  
4        179.0    200.0       86.0  


In [46]:
# Select all columns related to age distribution
age_columns = [col for col in df_census_s0101.columns 
               if col.startswith('Estimate!!Total!!Total population!!AGE!!')]

# Optional: preview them
print(age_columns)


['Estimate!!Total!!Total population!!AGE!!Under 5 years', 'Estimate!!Total!!Total population!!AGE!!5 to 9 years', 'Estimate!!Total!!Total population!!AGE!!10 to 14 years', 'Estimate!!Total!!Total population!!AGE!!15 to 19 years', 'Estimate!!Total!!Total population!!AGE!!20 to 24 years', 'Estimate!!Total!!Total population!!AGE!!25 to 29 years', 'Estimate!!Total!!Total population!!AGE!!30 to 34 years', 'Estimate!!Total!!Total population!!AGE!!35 to 39 years', 'Estimate!!Total!!Total population!!AGE!!40 to 44 years', 'Estimate!!Total!!Total population!!AGE!!45 to 49 years', 'Estimate!!Total!!Total population!!AGE!!50 to 54 years', 'Estimate!!Total!!Total population!!AGE!!55 to 59 years', 'Estimate!!Total!!Total population!!AGE!!60 to 64 years', 'Estimate!!Total!!Total population!!AGE!!65 to 69 years', 'Estimate!!Total!!Total population!!AGE!!70 to 74 years', 'Estimate!!Total!!Total population!!AGE!!75 to 79 years', 'Estimate!!Total!!Total population!!AGE!!80 to 84 years', 'Estimate!!Total

In [47]:
df_census_s0101['zip'] = df_census_s0101['Geographic Area Name'].str.extract(r'(\d{5})')

# Keep only zip and age columns
df_census_ages = df_census_s0101[['zip'] + age_columns].copy()


In [48]:
df_merged = pd.merge(df_restaurants, df_census_ages, on='zip', how='left')


In [49]:
# Step 4: Preview the merged dataset
# Show the first few rows with restaurant name, ZIP code, and some age-related columns

selected_columns = ['name', 'zip'] + age_columns[:5]  # show first 5 age groups for preview
print(df_merged[selected_columns].head())


                           name    zip  \
0           La Grande Boucherie  10019   
1                    The Modern  10019   
2                STK Steakhouse  10036   
3              Tony's Di Napoli  10036   
4  La Pecora Bianca Bryant Park  10018   

   Estimate!!Total!!Total population!!AGE!!Under 5 years  \
0                                             1383.0       
1                                             1383.0       
2                                              551.0       
3                                              551.0       
4                                              179.0       

   Estimate!!Total!!Total population!!AGE!!5 to 9 years  \
0                                              753.0      
1                                              753.0      
2                                              649.0      
3                                              649.0      
4                                              200.0      

   Estimate!!Total!!Total popul

In [50]:
for col in df_census_s0101.columns:
    print(col)


Geography
Geographic Area Name
Estimate!!Total!!Total population
Margin of Error!!Total!!Total population
Estimate!!Total!!Total population!!AGE!!Under 5 years
Margin of Error!!Total!!Total population!!AGE!!Under 5 years
Estimate!!Total!!Total population!!AGE!!5 to 9 years
Margin of Error!!Total!!Total population!!AGE!!5 to 9 years
Estimate!!Total!!Total population!!AGE!!10 to 14 years
Margin of Error!!Total!!Total population!!AGE!!10 to 14 years
Estimate!!Total!!Total population!!AGE!!15 to 19 years
Margin of Error!!Total!!Total population!!AGE!!15 to 19 years
Estimate!!Total!!Total population!!AGE!!20 to 24 years
Margin of Error!!Total!!Total population!!AGE!!20 to 24 years
Estimate!!Total!!Total population!!AGE!!25 to 29 years
Margin of Error!!Total!!Total population!!AGE!!25 to 29 years
Estimate!!Total!!Total population!!AGE!!30 to 34 years
Margin of Error!!Total!!Total population!!AGE!!30 to 34 years
Estimate!!Total!!Total population!!AGE!!35 to 39 years
Margin of Error!!Total!!To

In [51]:
df_merged.to_csv('combined_restaurants_enriched.csv', index=False)


In [52]:
df_merged.to_csv('combined_restaurants_enriched.csv', index=False)
print("✅ Enriched data saved to 'combined_restaurants_enriched.csv'")


✅ Enriched data saved to 'combined_restaurants_enriched.csv'
